# Tutorial \#2: LibKet - The Basics

In this second part of the tutorial you will learn to write simple quantum algorithms in **LibKet** and to execute them on the quantum computer simulator QuEST.

## Getting started

**LibKet** is implemented as header-only C++17 library. It suffices to include the main header file `LibKet.hpp` in your application.

In [ ]:
#include "LibKet.hpp"

**LibKet**'s main functionality resides in the global namespace `LibKet` and in the nested namespaces `circuits`, `filters`, and `gates`.

Let's import their functionality.

In [ ]:
using namespace LibKet;
using namespace LibKet::circuits;
using namespace LibKet::filters;
using namespace LibKet::gates;

Whenever you'd like to get help on a **LibKet** class or function you can simply type 
``?LibKet::QProgram`` to display the [documentation](https://libket.gitlab.io/LibKet/)

In [ ]:
?LibKet::QProgram

## Single-qubit quantum gates
Implementing and executing a quantum algorithm in **LibKet** consists of two steps
1. Creation of a generic hardware-agnostic **quantum program** or **quantum expression**
2. Evaluation of the program or expression on a concrete **quantum backend**

Since quantum programs are the more traditional approach of implementing quantum algorithms we will start with it and redo some of the examples using quantum expressions in the third part of the tutorial.

### Quantum program
Let's start with the first step and create a quantum program that initializes the qubit to zero, $ \lvert 0 \rangle$, and apply a **Pauli-X** or **NOT**-gate to it, $ \lvert\psi \rangle = X\lvert 0\rangle $.

In [ ]:
QProgram prog1;
prog1.x(0);

That's it. We can now proceed to step 2.

### Quantum backend
Let's evaluate our first quantum program on a quantum simulator.

**LibKet** supports a couple of them but the easiest one is probably the open-source [QuEST](https://quest.qtechtheory.org) *Quantum Exact Simulation Toolkit.* 

The following code snippet shows how to
1.  Create a quantum device for a single qubit
2.  Load our quantum program into it
3.  Evaluate the quantum kernel and print the result

In [ ]:
QDevice<QDeviceType::quest, 1> device1;
device1(prog1.to_string()).eval(1);
std::cout << device1.reg() << std::endl;

What we see here are the complex-valued coefficients $ \alpha $ and $ \beta $ of the state vector 

$$ \lvert\psi\rangle = \alpha \lvert 0 \rangle + \beta \lvert 1 \rangle $$

with $ |\alpha|^2+|\beta|^2 = 1 $. Since we started with $ \lvert 0 \rangle $ and applied a **NOT**-gate to it, we obtain 

$$ (0+i0)\lvert 0 \rangle + (1+i0)\lvert 1 \rangle = \lvert 1 \rangle $$

If you rerun the above code block you will obtain the error
```
input_line_22:2:33: error: redefinition of 'device1'
 QDevice<QDeviceType::quest, 1> device1;
                                ^
input_line_10:2:33: note: previous definition is here
 QDevice<QDeviceType::quest, 1> device1;
                                ^
```
This is expected C++ behavior as you try to redefine the object `device1`. That's why we number programs and devices in this tutorial. If you nontheless want to rerun code blocks just comment out the definitions, e.g.
```cpp
// QDevice<QDeviceType::quest, 1> device1;
device1(prog1.to_string()).eval(1);
std::cout << device1.reg() << std::endl;
```

#### Exercise 1
Try yourself to create an expression for $ X\lvert1\rangle $ and evaluate it on the QuEST simulator. What is the value of the coefficients?

### Hadamard gate

Let us now create the quantum expression for $ H\lvert0\rangle $, the **Hadamard** gate.

In [ ]:
QProgram prog3;
prog3.h(0);

QDevice<QDeviceType::quest, 1> device3;
device3(prog3.to_string()).eval(1);
std::cout << device3.reg() << std::endl;

In this case the state vector is 'halfway between' $ \lvert0\rangle $ and $ \lvert1\rangle $ since $ \alpha = \sqrt{2}+i0 $ and $ \beta = \sqrt{2}+i0 $. This phenomenon is termed **superposition** of the two pure states. It should be noted that this look behind the curtain is only possible with a quantum simulator but not with a real hardware device.

The coefficients $ \alpha $ and $ \beta$ are termed the **probability amplitudes** and $ \lvert\alpha\lvert^2 $ and $ \lvert\beta\lvert^2 $ are the **probabilities**. The QuEST simulator allows you to calculate them directly by calling ``device.probabilities()``.

In [ ]:
QProgram prog4;
prog4.h(0);

QDevice<QDeviceType::quest, 1> device4;
device4(prog4.to_string()).eval(1);
std::cout << device4.probabilities() << std::endl;

#### Exercise 2
Try yourself to create an expression for $ H \lvert 1 \rangle $ and evaluate it on the QuEST simulator. What is the value of the coefficients? What is their probabilities?

### Measurement
On a real quantum computer, one cannot readout the values of the state vector directly. The same applies to the probabilities. The way to obtain the latter is to perform **measurements** at the end of the quantum circuit and run it multiple times.

In [ ]:
QProgram prog6;
prog6.h(0);
prog6.measure(0);

QDevice<QDeviceType::quest, 1> device6;
device6(prog6.to_string());
for(int i=0; i<10; ++i) {
    device6.eval(1);
    std::cout << device6.creg() << std::endl;
}

You should see that there is a 50:50 change of ending up in either of the two states.

## Single-qubit quantum circuits

#### Exercise 3
Create a quantum program for the following circuit and evaluate it

![HZH](images/single_qubit_circuit_HZH.png)

Next create a quantum expression for the following circuit and evaluate it

![HXH](images/single_qubit_circuit_HXH.png)

## Multi-qubit quantum circuits
Creating quantum programs for circuits with multiple qubits follows the same principle. Just make sure you give the correct qubit numbers. We follow the convention adopted by many quantum programming libraries that the top-most qubit is qubit number 0, the one below is qubit number 1, etcetera.

![HI](images/multi_qubit_circuit_HI.png)

A quantum program for the above two-qubit quantum circuit can be constructed as follows

In [ ]:
QProgram prog9;
prog9.h(0);
prog9.i(1);

In fact, the last line is not necessary since the identity gate does not change the state of the qubits and can be left out.

#### Exercise 4
What is the result of evaluating it on the QuEST simulator? Don't forget to set the number of qubits to two.

## Multi-qubit quantum gates
In the above example the multi-qubit circuit was composed of two single-qubit gates, whereby the identidy gate was left out since it had no effecty.

**LibKet** also provides a couple of true multi-qubit gates like the **CNOT**-gate. Since it operates on two qubits simultaneously we have to pass two arguments to it. The first one is the **control qubit** and the second is the **target qubit**.

In [ ]:
QProgram prog11;
prog11.cnot(0, 1);

## Multi-qubit quantum circuit

#### Exercise 5
Create a quantum expression for the following two-qubit circuit which creates the [first Bell state](https://en.wikipedia.org/wiki/Bell_state) $ \lvert \beta_{00}\rangle = \frac{1}{\sqrt{2}}(\lvert 00 \rangle + \lvert 11 \rangle) $ and execute it on the QuEST simulator

![Bell](images/multi_qubit_circuit_HI_CNOT.png)

If you want to apply the same quantum gate to multiple qubits at the same time you can pass the qubit numbers as lists, e.g.

In [ ]:
QProgram prog13;
prog13.h({0,2});
prog13.cnot({0,2},{1,3});
prog13.measure({0,1,2,3});

## Self-study exercise: Bell states
Write quantum programs for the three other Bell states

$$ \lvert \beta_{01}\rangle = \frac{1}{\sqrt{2}}(\lvert 01 \rangle + \lvert 10 \rangle) \qquad
   \lvert \beta_{10}\rangle = \frac{1}{\sqrt{2}}(\lvert 00 \rangle - \lvert 11 \rangle) / \sqrt{2} \qquad
   \lvert \beta_{11}\rangle = \frac{1}{\sqrt{2}}(\lvert 01 \rangle - \lvert 10 \rangle) / \sqrt{2} $$

## Self-study exercise: Basic quantum circuits
Write quantum programs for the following circuits.

![Exercise](images/multi_qubit_circuit_exercises.png)

The second gate in the third exercise is a **SWAP**-gate, ``swap(qubit0, qubit1)`` which simply swaps the two qubits.

The last gate in the fourth exercise is termed **Toffoli** or **CCNOT**-gate, ``ccnot(ctrl0, ctrl1, target)`` which applies a **Pauli-X** or **NOT**-operation on the target qubit of both control qubits are true.